<a href="https://colab.research.google.com/github/madhurapi/Food-Demand-Forecast/blob/main/Food_Demand_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

import pandas as pd
pd.set_option('display.max_column',None)

import warnings
warnings.filterwarnings('ignore')

import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats 
import statsmodels.api as sm
import statsmodels.formula.api as smf

from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import chi2, chi2_contingency
from statsmodels.stats.multicomp import pairwise_tukeyhsd

from scipy.stats import pearsonr

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from IPython.display import Image

from math import sqrt
import sklearn

from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn.model_selection import cross_val_predict, train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler


from sklearn.linear_model import SGDRegressor


from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

from sklearn.feature_selection import RFE
from imblearn.over_sampling import SMOTE

from sklearn import tree
from sklearn import metrics
from sklearn.tree import DecisionTreeRegressor

import graphviz
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import scale

get_ipython().run_line_magic('matplotlib', 'inline')
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [ ]:
# loading data

In [ ]:
centre_info = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Food Demand csv/fulfilment_center_info.csv")
centre_info.head()

,center_id,city_code,region_code,center_type,op_area
0,11,679,56,TYPE_A,3.7
1,13,590,56,TYPE_B,6.7
2,124,590,56,TYPE_C,4.0
3,66,648,34,TYPE_A,4.1
4,94,632,34,TYPE_C,3.6


In [ ]:
meal_info = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Food Demand csv/meal_info.csv")
meal_info.head()

,meal_id,category,cuisine
0,1885,Beverages,Thai
1,1993,Beverages,Thai
2,2539,Beverages,Thai
3,1248,Beverages,Indian
4,2631,Beverages,Indian


In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Food Demand csv/train (2).csv")
train_data.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1379560,1,55,1885,136.83,152.29,0,0,177
1,1466964,1,55,1993,136.83,135.83,0,0,270
2,1346989,1,55,2539,134.86,135.86,0,0,189
3,1338232,1,55,2139,339.50,437.53,0,0,54
4,1448490,1,55,2631,243.50,242.50,0,0,40


In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Food Demand csv/test_QoiMO9B.csv")
test_data.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured
0,1028232,146,55,1885,158.11,159.11,0,0
1,1127204,146,55,1993,160.11,159.11,0,0
2,1212707,146,55,2539,157.14,159.14,0,0
3,1082698,146,55,2631,162.02,162.02,0,0
4,1400926,146,55,1248,163.93,163.93,0,0


In [ ]:
pivot_1 = meal_info.pivot_table(index =['cuisine'],
                       values =['meal_id'],
                       aggfunc ='count')
pivot_1

,meal_id
cuisine,
Continental,12
Indian,12
Italian,12
Thai,15


In [ ]:
# addition of calculated fields - dicount amt & %discount

In [ ]:
# train_data['%discount'] = ((train_data['base_price'] - train_data['checkout_price']) * 100) / train_data['base_price']
# train_data.head()

In [ ]:
# train_data['discount_amt'] = ((train_data['base_price'] - train_data['checkout_price']))
# train_data.head()

In [ ]:
# train_data['discount y/n'] = [1 if x > 0 else 0 for x in (train_data['base_price'] - train_data['checkout_price'])]
# train_data.head()

In [ ]:
train_data.isnull().sum()

id                       0
week                     0
center_id                0
meal_id                  0
checkout_price           0
base_price               0
emailer_for_promotion    0
homepage_featured        0
num_orders               0
dtype: int64

In [ ]:
test_data.isnull().sum()

id                       0
week                     0
center_id                0
meal_id                  0
checkout_price           0
base_price               0
emailer_for_promotion    0
homepage_featured        0
dtype: int64

In [ ]:
train_data.duplicated().sum()

0

In [ ]:
test_data.duplicated().sum()

0

In [ ]:
train_data.shape

(456548, 9)

In [ ]:
test_data.shape

(32573, 8)

In [ ]:
# dropping duplicate values
train_data.drop_duplicates(keep=False, inplace=True)
train_data.shape

(456548, 9)

In [ ]:
# dropping duplicate values
test_data.drop_duplicates(keep=False, inplace=True)
test_data.shape

(32573, 8)

In [ ]:
train_data.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders
0,1379560,1,55,1885,136.83,152.29,0,0,177
1,1466964,1,55,1993,136.83,135.83,0,0,270
2,1346989,1,55,2539,134.86,135.86,0,0,189
3,1338232,1,55,2139,339.50,437.53,0,0,54
4,1448490,1,55,2631,243.50,242.50,0,0,40


In [ ]:
one_hot_encoding_df = train_data[['center_id','meal_id']]

In [ ]:
one_hot_encoding_df_test = test_data[['center_id','meal_id']]

In [ ]:
one_hot_encoding_df.head()

,center_id,meal_id
0,55,1885
1,55,1993
2,55,2539
3,55,2139
4,55,2631


In [ ]:
one_hot_encoding_df_test.head()

,center_id,meal_id
0,55,1885
1,55,1993
2,55,2539
3,55,2631
4,55,1248


In [ ]:
for i in one_hot_encoding_df.columns:
  print(i, ':', (len(one_hot_encoding_df[i].unique()),'labels'))

center_id : (77, 'labels')
meal_id : (51, 'labels')


In [ ]:
#showing top 30 frequently meal_id and center_id
train_data.center_id.value_counts().sort_values(ascending = False).head(30)

13     7046
10     7015
52     6993
43     6970
67     6915
137    6894
174    6891
51     6880
27     6853
108    6843
11     6801
132    6788
36     6776
59     6716
89     6706
153    6696
104    6687
20     6671
30     6666
53     6538
23     6434
80     6396
126    6389
17     6333
109    6296
99     6293
72     6252
101    6172
146    6164
76     6099
Name: center_id, dtype: int64

In [ ]:
train_data.meal_id.value_counts().sort_values(ascending = False).head(30)

2290    11138
1062    11137
1727    11136
1109    11134
1754    11132
2707    11123
1778    11121
1993    11116
1962    11114
1971    11102
1885    11092
2581    11072
2826    11057
2322    11002
1558    10952
1207    10806
2640    10747
1230    10746
1878    10527
2631    10458
2539    10451
1543    10236
2760    10209
1248     9939
2664     9853
2704     9811
1445     9727
2126     9705
1216     9695
2304     9676
Name: meal_id, dtype: int64

In [ ]:
# for top 30 frequency labels
top_30 = [i for i in train_data.center_id.value_counts().sort_values(ascending = False).head(30)]
top_30

[7046,
 7015,
 6993,
 6970,
 6915,
 6894,
 6891,
 6880,
 6853,
 6843,
 6801,
 6788,
 6776,
 6716,
 6706,
 6696,
 6687,
 6671,
 6666,
 6538,
 6434,
 6396,
 6389,
 6333,
 6296,
 6293,
 6252,
 6172,
 6164,
 6099]

In [ ]:
topp_30 = [i for i in train_data.center_id.value_counts().sort_values(ascending = False).head(30).index]
topp_30#to get index

[13,
 10,
 52,
 43,
 67,
 137,
 174,
 51,
 27,
 108,
 11,
 132,
 36,
 59,
 89,
 153,
 104,
 20,
 30,
 53,
 23,
 80,
 126,
 17,
 109,
 99,
 72,
 101,
 146,
 76]

In [ ]:
topp_30_meal_id = [i for i in train_data.meal_id.value_counts().sort_values(ascending = False).head(30).index]
topp_30_meal_id#to get index

[2290,
 1062,
 1727,
 1109,
 1754,
 2707,
 1778,
 1993,
 1962,
 1971,
 1885,
 2581,
 2826,
 2322,
 1558,
 1207,
 2640,
 1230,
 1878,
 2631,
 2539,
 1543,
 2760,
 1248,
 2664,
 2704,
 1445,
 2126,
 1216,
 2304]

In [ ]:
for label in topp_30:
  train_data[label] = np.where(train_data['center_id'] == label, 1 , 0) #center_id

In [ ]:
# now getting dummies for a given variables with for loop instead of get dummies function.
#  bcoz for each variable we need to find the top 10 and then drop the other or merging it hence for loop will do it.

In [ ]:
# function to create the dummy variable for the variables
def one_hot_top_x(df, variable, top_x_labels):
  # top_x_labels is given bcoz we can change the top frequent labels if we want.

  for label in top_x_labels:
    df[label] = np.where(df[variable] == label, 1, 0)


one_hot_top_x(train_data, 'meal_id', topp_30_meal_id) # topp_30 is already defined which can be changed if needed.
train_data.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,13,10,52,43,67,137,174,51,27,108,11,132,36,59,89,153,104,20,30,53,23,80,126,17,109,99,72,101,146,76,2290,1062,1727,1109,1754,2707,1778,1993,1962,1971,1885,2581,2826,2322,1558,1207,2640,1230,1878,2631,2539,1543,2760,1248,2664,2704,1445,2126,1216,2304
0,1379560,1,55,1885,136.83,152.29,0,0,177,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1466964,1,55,1993,136.83,135.83,0,0,270,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1346989,1,55,2539,134.86,135.86,0,0,189,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,1338232,1,55,2139,339.50,437.53,0,0,54,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1448490,1,55,2631,243.50,242.50,0,0,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
one_hot_top_x(train_data, 'center_id', topp_30) # topp_30 is already defined which can be changed if needed.
train_data.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,13,10,52,43,67,137,174,51,27,108,11,132,36,59,89,153,104,20,30,53,23,80,126,17,109,99,72,101,146,76,2290,1062,1727,1109,1754,2707,1778,1993,1962,1971,1885,2581,2826,2322,1558,1207,2640,1230,1878,2631,2539,1543,2760,1248,2664,2704,1445,2126,1216,2304
0,1379560,1,55,1885,136.83,152.29,0,0,177,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1466964,1,55,1993,136.83,135.83,0,0,270,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1346989,1,55,2539,134.86,135.86,0,0,189,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,1338232,1,55,2139,339.50,437.53,0,0,54,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1448490,1,55,2631,243.50,242.50,0,0,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 456548 entries, 0 to 456547
Data columns (total 69 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     456548 non-null  int64  
 1   week                   456548 non-null  int64  
 2   center_id              456548 non-null  int64  
 3   meal_id                456548 non-null  int64  
 4   checkout_price         456548 non-null  float64
 5   base_price             456548 non-null  float64
 6   emailer_for_promotion  456548 non-null  int64  
 7   homepage_featured      456548 non-null  int64  
 8   num_orders             456548 non-null  int64  
 9   13                     456548 non-null  int64  
 10  10                     456548 non-null  int64  
 11  52                     456548 non-null  int64  
 12  43                     456548 non-null  int64  
 13  67                     456548 non-null  int64  
 14  137                    456548 non-nu

In [ ]:
train_data.drop(['center_id','meal_id'],axis = 1, inplace = True)

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 456548 entries, 0 to 456547
Data columns (total 67 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     456548 non-null  int64  
 1   week                   456548 non-null  int64  
 2   checkout_price         456548 non-null  float64
 3   base_price             456548 non-null  float64
 4   emailer_for_promotion  456548 non-null  int64  
 5   homepage_featured      456548 non-null  int64  
 6   num_orders             456548 non-null  int64  
 7   13                     456548 non-null  int64  
 8   10                     456548 non-null  int64  
 9   52                     456548 non-null  int64  
 10  43                     456548 non-null  int64  
 11  67                     456548 non-null  int64  
 12  137                    456548 non-null  int64  
 13  174                    456548 non-null  int64  
 14  51                     456548 non-nu

In [ ]:
test_with_dummies_center_id = pd.get_dummies(test_data['center_id'])
print(test_with_dummies_center_id)

       10   11   13   14   17   20   23   24   26   27   29   30   32   34   \
0        0    0    0    0    0    0    0    0    0    0    0    0    0    0   
1        0    0    0    0    0    0    0    0    0    0    0    0    0    0   
2        0    0    0    0    0    0    0    0    0    0    0    0    0    0   
3        0    0    0    0    0    0    0    0    0    0    0    0    0    0   
4        0    0    0    0    0    0    0    0    0    0    0    0    0    0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
32568    0    0    0    0    0    0    0    0    0    0    0    0    0    0   
32569    0    0    0    0    0    0    0    0    0    0    0    0    0    0   
32570    0    0    0    0    0    0    0    0    0    0    0    0    0    0   
32571    0    0    0    0    0    0    0    0    0    0    0    0    0    0   
32572    0    0    0    0    0    0    0    0    0    0    0    0    0    0   

       36   39   41   42   43   50   51   52   53  

In [ ]:
test_with_dummies_meal_id = pd.get_dummies(test_data['meal_id'])


In [ ]:
print(test_with_dummies_meal_id)

       1062  1109  1198  1207  1216  1230  1247  1248  1311  1438  1445  1525  \
0         0     0     0     0     0     0     0     0     0     0     0     0   
1         0     0     0     0     0     0     0     0     0     0     0     0   
2         0     0     0     0     0     0     0     0     0     0     0     0   
3         0     0     0     0     0     0     0     0     0     0     0     0   
4         0     0     0     0     0     0     0     1     0     0     0     0   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
32568     0     0     0     0     0     0     0     0     0     0     0     0   
32569     0     0     0     0     0     0     0     0     0     0     0     0   
32570     0     0     0     0     0     0     0     0     0     0     0     0   
32571     0     0     0     0     0     0     0     0     0     0     0     0   
32572     0     0     0     0     0     0     0     0     0     0     0     0   

       1543  1558  1571  17

In [ ]:
test_data1 = test_data.drop(['center_id','meal_id'],axis = 1, inplace = True)

In [ ]:
test_data.head()

,id,week,checkout_price,base_price,emailer_for_promotion,homepage_featured
0,1028232,146,158.11,159.11,0,0
1,1127204,146,160.11,159.11,0,0
2,1212707,146,157.14,159.14,0,0
3,1082698,146,162.02,162.02,0,0
4,1400926,146,163.93,163.93,0,0


In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32573 entries, 0 to 32572
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     32573 non-null  int64  
 1   week                   32573 non-null  int64  
 2   checkout_price         32573 non-null  float64
 3   base_price             32573 non-null  float64
 4   emailer_for_promotion  32573 non-null  int64  
 5   homepage_featured      32573 non-null  int64  
dtypes: float64(2), int64(4)
memory usage: 1.7 MB


In [ ]:
test_data = pd.concat([test_data,test_with_dummies_center_id,test_with_dummies_meal_id],axis =1)
test_data.head()

,id,week,checkout_price,base_price,emailer_for_promotion,homepage_featured,10,11,13,14,17,20,23,24,26,27,29,30,32,34,36,39,41,42,43,50,51,52,53,55,57,58,59,61,64,65,66,67,68,72,73,74,75,76,77,80,81,83,86,88,89,91,92,93,94,97,99,101,102,104,106,108,109,110,113,124,126,129,132,137,139,143,145,146,149,152,153,157,161,162,174,177,186,1062,1109,1198,1207,1216,1230,1247,1248,1311,1438,1445,1525,1543,1558,1571,1727,1754,1770,1778,1803,1847,1878,1885,1902,1962,1971,1993,2104,2126,2139,2290,2304,2306,2322,2444,2490,2492,2494,2539,2569,2577,2581,2631,2640,2664,2704,2707,2760,2826,2867,2956
0,1028232,146,158.11,159.11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1127204,146,160.11,159.11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1212707,146,157.14,159.14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,1082698,146,162.02,162.02,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,1400926,146,163.93,163.93,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32573 entries, 0 to 32572
Columns: 134 entries, id to 2956
dtypes: float64(2), int64(4), uint8(128)
memory usage: 5.7 MB


In [ ]:
to_be_scaled= train_data[['base_price','checkout_price']]
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(to_be_scaled)
df1_scaled= pd.DataFrame(scaled_data, columns= to_be_scaled.columns)

In [ ]:
to_be_scaled_test = test_data[['base_price','checkout_price']]
from sklearn.preprocessing import StandardScaler
scaler_test = StandardScaler()
scaled_data_test = scaler_test.fit_transform(to_be_scaled_test)
df1_scaled_test= pd.DataFrame(scaled_data_test, columns= to_be_scaled_test.columns)

In [ ]:
train_data.drop(to_be_scaled, axis=1, inplace=True)
train_data.head()

,id,week,emailer_for_promotion,homepage_featured,num_orders,13,10,52,43,67,137,174,51,27,108,11,132,36,59,89,153,104,20,30,53,23,80,126,17,109,99,72,101,146,76,2290,1062,1727,1109,1754,2707,1778,1993,1962,1971,1885,2581,2826,2322,1558,1207,2640,1230,1878,2631,2539,1543,2760,1248,2664,2704,1445,2126,1216,2304
0,1379560,1,0,0,177,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1466964,1,0,0,270,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1346989,1,0,0,189,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,1338232,1,0,0,54,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1448490,1,0,0,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_data.drop(to_be_scaled_test, axis =1, inplace = True)
test_data.head()

,id,week,emailer_for_promotion,homepage_featured,10,11,13,14,17,20,23,24,26,27,29,30,32,34,36,39,41,42,43,50,51,52,53,55,57,58,59,61,64,65,66,67,68,72,73,74,75,76,77,80,81,83,86,88,89,91,92,93,94,97,99,101,102,104,106,108,109,110,113,124,126,129,132,137,139,143,145,146,149,152,153,157,161,162,174,177,186,1062,1109,1198,1207,1216,1230,1247,1248,1311,1438,1445,1525,1543,1558,1571,1727,1754,1770,1778,1803,1847,1878,1885,1902,1962,1971,1993,2104,2126,2139,2290,2304,2306,2322,2444,2490,2492,2494,2539,2569,2577,2581,2631,2640,2664,2704,2707,2760,2826,2867,2956
0,1028232,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1127204,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1212707,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,1082698,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,1400926,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_data = pd.concat([df1_scaled,train_data],axis=1)
train_data.head()

,base_price,checkout_price,id,week,emailer_for_promotion,homepage_featured,num_orders,13,10,52,43,67,137,174,51,27,108,11,132,36,59,89,153,104,20,30,53,23,80,126,17,109,99,72,101,146,76,2290,1062,1727,1109,1754,2707,1778,1993,1962,1971,1885,2581,2826,2322,1558,1207,2640,1230,1878,2631,2539,1543,2760,1248,2664,2704,1445,2126,1216,2304
0,-1.256048,-1.277687,1379560,1,0,0,177,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-1.358465,-1.277687,1466964,1,0,0,270,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-1.358278,-1.290568,1346989,1,0,0,189,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0.518763,0.047477,1338232,1,0,0,54,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-0.694746,-0.580222,1448490,1,0,0,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_data = pd.concat([df1_scaled_test, test_data],axis =1)
test_data.head()

,base_price,checkout_price,id,week,emailer_for_promotion,homepage_featured,10,11,13,14,17,20,23,24,26,27,29,30,32,34,36,39,41,42,43,50,51,52,53,55,57,58,59,61,64,65,66,67,68,72,73,74,75,76,77,80,81,83,86,88,89,91,92,93,94,97,99,101,102,104,106,108,109,110,113,124,126,129,132,137,139,143,145,146,149,152,153,157,161,162,174,177,186,1062,1109,1198,1207,1216,1230,1247,1248,1311,1438,1445,1525,1543,1558,1571,1727,1754,1770,1778,1803,1847,1878,1885,1902,1962,1971,1993,2104,2126,2139,2290,2304,2306,2322,2444,2490,2492,2494,2539,2569,2577,2581,2631,2640,2664,2704,2707,2760,2826,2867,2956
0,-1.272230,-1.193987,1028232,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-1.272230,-1.180991,1127204,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-1.272037,-1.200290,1212707,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,-1.253474,-1.168579,1082698,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,-1.241163,-1.156168,1400926,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32573 entries, 0 to 32572
Columns: 134 entries, base_price to 2956
dtypes: float64(2), int64(4), uint8(128)
memory usage: 5.5 MB


In [ ]:
#need to drop those extra index of centerid and meal id. then predictions to be done

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456548 entries, 0 to 456547
Data columns (total 67 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   base_price             456548 non-null  float64
 1   checkout_price         456548 non-null  float64
 2   id                     456548 non-null  int64  
 3   week                   456548 non-null  int64  
 4   emailer_for_promotion  456548 non-null  int64  
 5   homepage_featured      456548 non-null  int64  
 6   num_orders             456548 non-null  int64  
 7   13                     456548 non-null  int64  
 8   10                     456548 non-null  int64  
 9   52                     456548 non-null  int64  
 10  43                     456548 non-null  int64  
 11  67                     456548 non-null  int64  
 12  137                    456548 non-null  int64  
 13  174                    456548 non-null  int64  
 14  51                     456548 non-nu

In [ ]:
test_data.head()

,base_price,checkout_price,id,week,emailer_for_promotion,homepage_featured,10,11,13,14,17,20,23,24,26,27,29,30,32,34,36,39,41,42,43,50,51,52,53,55,57,58,59,61,64,65,66,67,68,72,73,74,75,76,77,80,81,83,86,88,89,91,92,93,94,97,99,101,102,104,106,108,109,110,113,124,126,129,132,137,139,143,145,146,149,152,153,157,161,162,174,177,186,1062,1109,1198,1207,1216,1230,1247,1248,1311,1438,1445,1525,1543,1558,1571,1727,1754,1770,1778,1803,1847,1878,1885,1902,1962,1971,1993,2104,2126,2139,2290,2304,2306,2322,2444,2490,2492,2494,2539,2569,2577,2581,2631,2640,2664,2704,2707,2760,2826,2867,2956
0,-1.272230,-1.193987,1028232,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-1.272230,-1.180991,1127204,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-1.272037,-1.200290,1212707,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,-1.253474,-1.168579,1082698,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,-1.241163,-1.156168,1400926,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
train_data.columns

Index([           'base_price',        'checkout_price',
                          'id',                  'week',
       'emailer_for_promotion',     'homepage_featured',
                  'num_orders',                      13,
                            10,                      52,
                            43,                      67,
                           137,                     174,
                            51,                      27,
                           108,                      11,
                           132,                      36,
                            59,                      89,
                           153,                     104,
                            20,                      30,
                            53,                      23,
                            80,                     126,
                            17,                     109,
                            99,                      72,
                           101,

In [ ]:
test_data2 = test_data.copy(deep = True)

In [ ]:
test_data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32573 entries, 0 to 32572
Columns: 134 entries, base_price to 2956
dtypes: float64(2), int64(4), uint8(128)
memory usage: 5.5 MB


In [ ]:
test_with_dummies_center_id.columns

Int64Index([ 10,  11,  13,  14,  17,  20,  23,  24,  26,  27,  29,  30,  32,
             34,  36,  39,  41,  42,  43,  50,  51,  52,  53,  55,  57,  58,
             59,  61,  64,  65,  66,  67,  68,  72,  73,  74,  75,  76,  77,
             80,  81,  83,  86,  88,  89,  91,  92,  93,  94,  97,  99, 101,
            102, 104, 106, 108, 109, 110, 113, 124, 126, 129, 132, 137, 139,
            143, 145, 146, 149, 152, 153, 157, 161, 162, 174, 177, 186],
           dtype='int64')

In [ ]:
test_with_dummies_meal_id.columns

Int64Index([1062, 1109, 1198, 1207, 1216, 1230, 1247, 1248, 1311, 1438, 1445,
            1525, 1543, 1558, 1571, 1727, 1754, 1770, 1778, 1803, 1847, 1878,
            1885, 1902, 1962, 1971, 1993, 2104, 2126, 2139, 2290, 2304, 2306,
            2322, 2444, 2490, 2492, 2494, 2539, 2569, 2577, 2581, 2631, 2640,
            2664, 2704, 2707, 2760, 2826, 2867, 2956],
           dtype='int64')

In [ ]:
test_data2.head()

,base_price,checkout_price,id,week,emailer_for_promotion,homepage_featured,10,11,13,14,17,20,23,24,26,27,29,30,32,34,36,39,41,42,43,50,51,52,53,55,57,58,59,61,64,65,66,67,68,72,73,74,75,76,77,80,81,83,86,88,89,91,92,93,94,97,99,101,102,104,106,108,109,110,113,124,126,129,132,137,139,143,145,146,149,152,153,157,161,162,174,177,186,1062,1109,1198,1207,1216,1230,1247,1248,1311,1438,1445,1525,1543,1558,1571,1727,1754,1770,1778,1803,1847,1878,1885,1902,1962,1971,1993,2104,2126,2139,2290,2304,2306,2322,2444,2490,2492,2494,2539,2569,2577,2581,2631,2640,2664,2704,2707,2760,2826,2867,2956
0,-1.272230,-1.193987,1028232,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,-1.272230,-1.180991,1127204,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-1.272037,-1.200290,1212707,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,-1.253474,-1.168579,1082698,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,-1.241163,-1.156168,1400926,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_data2.drop([14, 24,  26,  29, 32],axis =1, inplace =True)


In [ ]:
test_data2.drop([34,  39,  41,  42,  50, 55,  57,  58,61,  64,  65,  66,  68, 73,  74,  75,  77,
               81,  83,  86,  88,  91,  92,  93,  94,  97,102, 106, 110, 113, 124, 129, 139,143, 145, 149, 152, 157, 161, 
               162, 177, 186,1198, 1247, 1311, 1438,1525, 1571, 1770, 1803, 1847,1902, 2104, 2139, 2306,2444, 2490, 2492, 2494, 2569, 2577,
              2867, 2956],axis =1, inplace =True)

In [ ]:
test_data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32573 entries, 0 to 32572
Data columns (total 66 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   base_price             32573 non-null  float64
 1   checkout_price         32573 non-null  float64
 2   id                     32573 non-null  int64  
 3   week                   32573 non-null  int64  
 4   emailer_for_promotion  32573 non-null  int64  
 5   homepage_featured      32573 non-null  int64  
 6   10                     32573 non-null  uint8  
 7   11                     32573 non-null  uint8  
 8   13                     32573 non-null  uint8  
 9   17                     32573 non-null  uint8  
 10  20                     32573 non-null  uint8  
 11  23                     32573 non-null  uint8  
 12  27                     32573 non-null  uint8  
 13  30                     32573 non-null  uint8  
 14  36                     32573 non-null  uint8  
 15  43

In [ ]:
X = train_data.iloc[:,[0,1,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,
                       31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,
                       57,58,59,60,61,62,63,64,65,66]]
y = train_data.iloc[:,6]
print(y)
print(X.info())

0         177
1         270
2         189
3          54
4          40
         ... 
456543     68
456544     42
456545    501
456546    729
456547    162
Name: num_orders, Length: 456548, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456548 entries, 0 to 456547
Data columns (total 65 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   base_price             456548 non-null  float64
 1   checkout_price         456548 non-null  float64
 2   week                   456548 non-null  int64  
 3   emailer_for_promotion  456548 non-null  int64  
 4   homepage_featured      456548 non-null  int64  
 5   13                     456548 non-null  int64  
 6   10                     456548 non-null  int64  
 7   52                     456548 non-null  int64  
 8   43                     456548 non-null  int64  
 9   67                     456548 non-null  int64  
 10  137                    456548 non-null  int64  

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 42)

In [ ]:
# n_estimators = [5,20,50,100] # number of trees in the random forest
# max_features = ['auto', 'sqrt'] # number of features in consideration at every split
# max_depth = [int(x) for x in np.linspace(10, 120, num = 12)] # maximum number of levels allowed in each decision tree
# min_samples_split = [2, 6, 10] # minimum sample number to split a node
# min_samples_leaf = [1, 3, 4] # minimum sample number that can be stored in a leaf node
# bootstrap = [True, False] # method used to sample data points

# random_grid = {'n_estimators': n_estimators,

# 'max_features': max_features,

# 'max_depth': max_depth,

# 'min_samples_split': min_samples_split,

# 'min_samples_leaf': min_samples_leaf,

# 'bootstrap': bootstrap}

In [ ]:
## Importing Random Forest Classifier from the sklearn.ensemble
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# rf_random = RandomizedSearchCV(estimator = rf,param_distributions = random_grid,
#                n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)

In [ ]:
rf.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
y_pred_rf = rf.predict(X_test)

In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32573 entries, 0 to 32572
Columns: 134 entries, base_price to 2956
dtypes: float64(2), int64(4), uint8(128)
memory usage: 5.5 MB


In [ ]:
test_data2.drop(['id'],axis =1,inplace =True)

In [ ]:
predictions1 = rf.predict(test_data2)

In [ ]:
print('R Square:',rf.score(X_test, y_test))
print('RMSLE:', 100*np.sqrt(metrics.mean_squared_log_error(y_test, y_pred_rf)))

R Square: 0.7641377498671311
RMSLE: 64.62085703786404


In [ ]:
y_pred_rf_train = rf.predict(X_train)

In [ ]:
print('R Square:',rf.score(X_train, y_train))
print('RMSLE:', 100*np.sqrt(metrics.mean_squared_log_error(y_train, y_pred_rf_train)))

R Square: 0.9580808153544353
RMSLE: 38.766941071265194


In [ ]:
prediction_food = pd.DataFrame(predictions1)

In [ ]:
test_data3 =pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Food Demand csv/test_QoiMO9B.csv")
test_data3.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured
0,1028232,146,55,1885,158.11,159.11,0,0
1,1127204,146,55,1993,160.11,159.11,0,0
2,1212707,146,55,2539,157.14,159.14,0,0
3,1082698,146,55,2631,162.02,162.02,0,0
4,1400926,146,55,1248,163.93,163.93,0,0


In [ ]:
test_data3.columns

Index(['id', 'week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured'],
      dtype='object')

In [ ]:
test_data3.drop(['week', 'center_id', 'meal_id', 'checkout_price', 'base_price',
       'emailer_for_promotion', 'homepage_featured'],axis =1, inplace =True)

In [ ]:
final_df = pd.concat((test_data3,prediction_food),axis = 1 )
final_df.head()


,id,0
0,1028232,270.42
1,1127204,275.39
2,1212707,330.81
3,1082698,91.44
4,1400926,324.04


In [ ]:
final_df.rename(columns = {0:'num_orders'}, inplace = True)
food_demand_csv1 = final_df.to_csv('food_demand_csv1.csv', index = False)

In [ ]:
# n_estimators = [50,100] # number of trees in the random forest
# max_features = ['auto', 'sqrt'] # number of features in consideration at every split
# max_depth = [5,8,10] # maximum number of levels allowed in each decision tree
# min_samples_split = [2, 6] # minimum sample number to split a node
# min_samples_leaf = [1, 3] # minimum sample number that can be stored in a leaf node
# bootstrap = [True] # method used to sample data points

# random_grid = {'n_estimators': n_estimators,

# 'max_features': max_features,

# 'max_depth': max_depth,

# 'min_samples_split': min_samples_split,

# 'min_samples_leaf': min_samples_leaf,

# 'bootstrap': bootstrap}

In [ ]:

# ## Importing Random Forest Classifier from the sklearn.ensemble
# from sklearn.ensemble import RandomForestRegressor
# rf1 = RandomForestRegressor()

# from sklearn.model_selection import RandomizedSearchCV
# rf_random1 = RandomizedSearchCV(estimator = rf,param_distributions = random_grid,
#                n_iter = 100, cv = 5, verbose=2, random_state=35, n_jobs = -1)


# rf_random1.fit(X_train, y_train)
# #predictions
# y_pred_rf = rf.predict(X_test)
# #test score
# print('R Square_test:',rf.score(X_test, y_test))
# print('RMSLE_test:', 100*np.sqrt(metrics.mean_squared_log_error(y_test, y_pred_rf)))
# #train_predictions
# y_pred_rf_train = rf.predict(X_train)

# print('R Square_train:',rf.score(X_train, y_train))
# print('RMSLE_train:', 100*np.sqrt(metrics.mean_squared_log_error(y_train, y_pred_rf_train)))

In [ ]:


# ## Importing Random Forest Classifier from the sklearn.ensemble
# from sklearn.ensemble import RandomForestRegressor
# rf1 = RandomForestRegressor()

# from sklearn.model_selection import RandomizedSearchCV
# rf_random1 = RandomizedSearchCV(estimator = rf,param_distributions = random_grid,
#                n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)


# rf_random1.fit(X_train, y_train)
# #predictions
# y_pred_rf = rf.predict(X_test)
# #test score
# print('R Square_test:',rf.score(X_test, y_test))
# print('RMSLE_test:', 100*np.sqrt(metrics.mean_squared_log_error(y_test, y_pred_rf)))
# #train_predictions
# y_pred_rf_train = rf.predict(X_train)

# print('R Square_train:',rf.score(X_train, y_train))
# print('RMSLE_train:', 100*np.sqrt(metrics.mean_squared_log_error(y_train, y_pred_rf_train)))

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456548 entries, 0 to 456547
Data columns (total 67 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   base_price             456548 non-null  float64
 1   checkout_price         456548 non-null  float64
 2   id                     456548 non-null  int64  
 3   week                   456548 non-null  int64  
 4   emailer_for_promotion  456548 non-null  int64  
 5   homepage_featured      456548 non-null  int64  
 6   num_orders             456548 non-null  int64  
 7   13                     456548 non-null  int64  
 8   10                     456548 non-null  int64  
 9   52                     456548 non-null  int64  
 10  43                     456548 non-null  int64  
 11  67                     456548 non-null  int64  
 12  137                    456548 non-null  int64  
 13  174                    456548 non-null  int64  
 14  51                     456548 non-nu

In [ ]:
XX = train_data.iloc[:,[0,1,3,4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,
                       31,32,33,34,35,36]]
yy = train_data.iloc[:,6]
print(yy)
print(XX.info())

from sklearn.model_selection import train_test_split
XX_train, XX_test, yy_train, yy_test = train_test_split(XX,yy,test_size = 0.3, random_state = 42)

rf2 = RandomForestRegressor()

rf2.fit(XX_train, yy_train)

y_pred_rf = rf2.predict(XX_test)

print('R Square:',rf2.score(XX_test, yy_test))
print('RMSLE:', 100*np.sqrt(metrics.mean_squared_log_error(yy_test, y_pred_rf)))

#train_predictions
y_pred_rf_train = rf2.predict(XX_train)

print('R Square_train:',rf2.score(XX_train, yy_train))
print('RMSLE_train:', 100*np.sqrt(metrics.mean_squared_log_error(yy_train, y_pred_rf_train)))

0         177
1         270
2         189
3          54
4          40
         ... 
456543     68
456544     42
456545    501
456546    729
456547    162
Name: num_orders, Length: 456548, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456548 entries, 0 to 456547
Data columns (total 35 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   base_price             456548 non-null  float64
 1   checkout_price         456548 non-null  float64
 2   week                   456548 non-null  int64  
 3   emailer_for_promotion  456548 non-null  int64  
 4   homepage_featured      456548 non-null  int64  
 5   13                     456548 non-null  int64  
 6   10                     456548 non-null  int64  
 7   52                     456548 non-null  int64  
 8   43                     456548 non-null  int64  
 9   67                     456548 non-null  int64  
 10  137                    456548 non-null  int64  

KeyboardInterrupt: ignored

In [ ]:
import xgboost as xg
xgb_r = xg.XGBRegressor(max_depth = 7,
                  n_estimators = 10, seed = 123)
# Fitting the model
xgb_r.fit(X_train, y_train)

# Predict the model
pred_xg = xgb_r.predict(X_test)


# RMSE Computation
rmse = np.sqrt(mean_squared_error(y_test, pred_xg))
print("RMSE : % f" %(rmse))

print('R Square_test:',xgb_r.score(X_test, y_test))
print('RMSLE_test:', 100*np.sqrt(metrics.mean_squared_log_error(y_test, pred_xg)))

#train_predictions
y_pred_xg_train = xgb_r.predict(X_train)

print('R Square_train:',xgb_r.score(X_train, y_train))
print('RMSLE_train:', 100*np.sqrt(metrics.mean_squared_log_error(y_train, y_pred_xg_train)))

[07:08:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE :  297.907030
R Square_test: 0.4248483988432452
RMSLE_test: 86.30575194128247
R Square_train: 0.4453080844967856
RMSLE_train: 85.9749539521134


In [ ]:
# #actual predictions
# predictions2 = xgb_r.predict(test_data2)
# prediction_food2 = pd.DataFrame(predictions2)
# final_df1 = pd.concat((test_data3,prediction_food2),axis = 1 )
# final_df1.head()
# final_df1.rename(columns = {0:'num_orders'}, inplace = True)
# food_demand_csv2 = final_df.to_csv('food_demand_csv2.csv', index = False)

In [ ]:
import lightgbm as lgb
model = lgb.LGBMRegressor()
model.fit(X_train, y_train)


# Predict the model
pred_lgb = model.predict(X_test)

# RMSE Computation
rmse = np.sqrt(mean_squared_error(y_test, pred_lgb))
print("RMSE : % f" %(rmse))

print('R Square_test:',model.score(X_test, y_test))
print('RMSLE_test:', 100*np.sqrt(metrics.mean_squared_log_error(y_test, pred_lgb)))

# #train_predictions
# y_pred_lgb_train = model.predict(X_train)

# print('R Square_train:',model.score(X_train, y_train))
# print('RMSLE_train:', 100*np.sqrt(metrics.mean_squared_log_error(y_train, y_pred_lgb_train)))

In [ ]:
%pip install catboost



In [ ]:
from catboost import CatBoostRegressor
# model1 = CatBoostRegressor()
model1 = CatBoostRegressor()
model1.fit(X_train, y_train)


# Predict the model
pred_1 = model1.predict(X_test)

# predictions2 = xgb_r.predict(test_data2)
# prediction_food2 = pd.DataFrame(predictions2)
# final_df1 = pd.concat((test_data3,prediction_food2),axis = 1 )
# final_df1.head()
# final_df1.rename(columns = {0:'num_orders'}, inplace = True)
# food_demand_csv2 = final_df.to_csv('food_demand_csv2.csv', index = False)

# RMSE Computation
rmse = np.sqrt(mean_squared_error(y_test, pred_1))
print("RMSE : % f" %(rmse))

print('R Square_test:',model1.score(X_test, y_test))
# print('RMSLE_test:', 100*np.sqrt(metrics.mean_squared_log_error(y_test, pred_1)))

#train_predictions
# y_pred_cbr_train = model1.predict(X_train)

# print('R Square_train:',model1.score(X_train, y_train))
# print('RMSLE_train:', 100*np.sqrt(metrics.mean_squared_log_error(y_train, y_pred_cbr_train)))

Learning rate set to 0.101837
0:	learn: 383.0809655	total: 56.9ms	remaining: 56.8s
1:	learn: 371.0335334	total: 100ms	remaining: 49.9s
2:	learn: 360.8148293	total: 148ms	remaining: 49s
3:	learn: 352.1993472	total: 189ms	remaining: 47s
4:	learn: 344.1950089	total: 229ms	remaining: 45.5s
5:	learn: 337.5783698	total: 269ms	remaining: 44.6s
6:	learn: 331.8187220	total: 312ms	remaining: 44.3s
7:	learn: 326.8873363	total: 352ms	remaining: 43.7s
8:	learn: 322.3934347	total: 409ms	remaining: 45s
9:	learn: 318.2155138	total: 449ms	remaining: 44.5s
10:	learn: 314.3485518	total: 488ms	remaining: 43.9s
11:	learn: 310.6151364	total: 532ms	remaining: 43.8s
12:	learn: 307.5269754	total: 572ms	remaining: 43.5s
13:	learn: 304.4555294	total: 616ms	remaining: 43.4s
14:	learn: 301.6420066	total: 657ms	remaining: 43.1s
15:	learn: 299.1650510	total: 694ms	remaining: 42.7s
16:	learn: 295.9197785	total: 741ms	remaining: 42.9s
17:	learn: 293.8971891	total: 779ms	remaining: 42.5s
18:	learn: 290.9987274	total: 8

In [ ]:
# train_predictions
y_pred_cbr_train = model1.predict(X_train)

print('R Square_train:',model1.score(X_train, y_train))